[View in Colaboratory](https://colab.research.google.com/github/Zulkhuu/head-gesture-recogniton/blob/master/python/evaluation/evaluation_colab.ipynb)

# Show system specification

In [10]:
from tensorflow.python.client import device_lib

print("Show System RAM Memory:\n\n")
!cat /proc/meminfo | egrep "MemTotal*"

print("\n\nShow Devices:\n\n"+str(device_lib.list_local_devices()))

Show System RAM Memory:


MemTotal:       13335268 kB


Show Devices:

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8108905546929401649
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14661938599781214009
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12059715898344616955
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15601436142556093201
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


# Download Dependencies to Google Colab Environment

This may take several minutes but needs to only be run once per session. Every Google Colab session has temporary storage for assets such as images and other scripts. When the session is closed these objects will be removed but the notebook will remain intact.

*Run the cell below to download necessary dependencies. After this is complete you can navigate to the FILES tab on the left and select refresh - you should see the files listed.*

In [11]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download dataset
Download Face Detection Data Set and Benchmark (FDDB) from Google Drive and unzip.
Original dataset link: http://vis-www.cs.umass.edu/fddb/index.html

In [0]:
# FDDB annotation files id=1HBIPoRFYK763kgOeHG011NAYWdox1Q2Z
download = drive.CreateFile({'id': '1HBIPoRFYK763kgOeHG011NAYWdox1Q2Z'})
download.GetContentFile('FDDB-folds.tgz')
!mkdir -p dataset/FDDB/FDDB-folds
!tar xzf FDDB-folds.tgz -C dataset/FDDB/

# FDDB image files id=1uEv2I4svspLRwvU-nMwQr7ZL1m5RhIYh
download = drive.CreateFile({'id': '1uEv2I4svspLRwvU-nMwQr7ZL1m5RhIYh'})
download.GetContentFile('originalPics.tar.gz')
!mkdir -p dataset/FDDB/originalPics
!tar xzf originalPics.tar.gz -C dataset/FDDB/originalPics/


#Run Face Detection on dataset

In [19]:
!pip install mtcnn
import cv2
import numpy as np
import pandas as pd
import itertools
import progressbar
import time
from mtcnn.mtcnn import MTCNN

detector = MTCNN()

def face_detect(image):
    """
    input: opencv image
    return: <left_x top_y width height detection_score>
    """
    dets = detector.detect_faces(image)
    results = []
    for det in dets:
        results.append('{:.6f} {:.6f} {:.6f} {:.6f} {:.6f}'.format(det['box'][0],det['box'][1],det['box'][2],det['box'][3],det['confidence']))
    return results

#Test FDDB dataset which is placed in ../dataset/FDDB
def test_fddb():
    database_path = 'dataset/FDDB'
    db_image_path = '{:s}/originalPics'.format(database_path)
    db_annotation_path = '{:s}/FDDB-folds'.format(database_path)
    n_fold = 10
    for i in range(n_fold):
        list_filename = '{:s}/FDDB-fold-{:02d}.txt'.format(db_annotation_path, i+1)
        output_filename = '{:s}/FDDB-fold-{:02d}-result.txt'.format(db_annotation_path, i+1)
        output_file = open(output_filename,mode='w')
        print('\r({:d}/{:d}) Detecting faces from image list in {:s} file'.format(i+1, n_fold, list_filename))
        with progressbar.ProgressBar(maxval=len(open(list_filename).readlines())) as bar:
            n = 0
            with open(list_filename) as f:
                for image_filename in f:
                    bar.update(n)
                    n += 1
                    full_filename = '{:s}/{:s}.jpg'.format(db_image_path,image_filename.rstrip())
                    output_file.write(image_filename)
                    image_cv2 = cv2.imread(full_filename)
                    results = face_detect(image_cv2)
                    output_file.write('{:d}\n'.format(len(results)))
                    for result in results:
                        output_file.write('{:s}\n'.format(result))

test_fddb()

N/A% (0 of 290) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

(1/10) Detecting faces from image list in dataset/FDDB/FDDB-folds/FDDB-fold-01.txt file


100% (290 of 290) |######################| Elapsed Time: 0:00:30 Time:  0:00:30
  0% (1 of 285) |                        | Elapsed Time: 0:00:00 ETA:   0:00:42

(2/10) Detecting faces from image list in dataset/FDDB/FDDB-folds/FDDB-fold-02.txt file


100% (285 of 285) |######################| Elapsed Time: 0:00:18 Time:  0:00:18
  0% (2 of 274) |                        | Elapsed Time: 0:00:00 ETA:   0:00:20

...

100% (276 of 276) |######################| Elapsed Time: 0:00:14 Time:  0:00:14
  1% (3 of 259) |                        | Elapsed Time: 0:00:00 ETA:   0:00:12

(9/10) Detecting faces from image list in dataset/FDDB/FDDB-folds/FDDB-fold-09.txt file


100% (259 of 259) |######################| Elapsed Time: 0:00:13 Time:  0:00:13
  0% (2 of 280) |                        | Elapsed Time: 0:00:00 ETA:   0:00:16

(10/10) Detecting faces from image list in dataset/FDDB/FDDB-folds/FDDB-fold-10.txt file


100% (280 of 280) |######################| Elapsed Time: 0:00:14 Time:  0:00:14
